# Machine Learning Exercise

Now it's time to practice what you have seen in the previous notebooks. Your task for today is to download the data from the database and train a model in order to predict if a patient has a heart disease or not. 

![](https://www.nicepng.com/png/detail/397-3975460_disease-high-quality-png-heart-disease-cartoon-png.png)

## Task:

1. Import the data from the database. The schema is called `heart`. You can use DBeaver to get an overview over the different tables and think about a good way to join them. 
2. Conduct a brief EDA to become familiar with the data. 
3. Preprocess the data as far as you need it and...
4. ...train a logistic regression model.

## What you should use/keep in mind:
 
* **Scale your data:** Which scaler works best in your case?
* **Tune your model:** Tune the hyperparameter of your model. You can start with a larger parameter grid and a `RandomizedSearchCV` and continue with a narrower parameter grid for your `GridSearchCV`.
* **Choose the right evaluation metric!**


## Data Overview

| column | additional information |
|--------|------------------------|
| age | age of patient |
| sex | gender of patient |
| chest_pain_type  | 1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic | 
| resting_blood_pressure |  | 
| fasting_blood_sugar | > 120 mg/dl, 1 = true, 0 = false | 
| thal | 0 = normal, 1 = fixed defect, 2 = reversable defect
| serum_cholestoral | in mg/dl | 
| resting_electrocardiographic_results | 0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria | 
| maximum_heartrate_achieved | | 
| exercise_induced_angina | 1 = yes, 0 = no | 
| oldpeak | ST depression induced by exercise relative to rest | 
| slope_of_the_peak_exercise_st_segment | 1= upsloping, 2 = flat, 3 = downsloping | 
| number_of_major_vessels_colored_by_flourosopy | |
| real_data | tag to distinguish between real and made up data | 
| heart_attack | 0 = little risk of heart attack, 1 = high risk of heart attack | 

## Import

In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

import numpy as np
import pandas as pd

from scipy.stats import skew
from scipy import stats
from scipy.stats import norm
from timeit import default_timer as timer


# Modelling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import loguniform

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.filterwarnings('ignore')

RSEED = 42


# Feel free to add all the libraries you need

## Getting the Data

The data for this exercise is stored in our postgres database in the schema `heart`. The different features are split thematically into five different tables. Your first task will be to have a look at the tables (e.g. in DBeaver) and figure out a way to join the information you need. As soon as you're happy with your query, you can use the following code cells to import the data into this notebook. 

In previous notebooks you've seen two different approaches to import data from a database into a notebook. The following code will use `sqlalchemy`in combination with pandas `pd.read_sql()` function. For the code to work, you need to copy the `.env` file from the previous repositories into this repository and change the query_string to your own query.

In [ ]:
# Read database string from .env file (no need to change anything)
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

db = create_engine(DB_STRING)

In [ ]:
# Define query to download data (add your query here)
query_string = ''' 
set SCHEMA 'heart';
SELECT p.id as patient_id,p.age,p.sex, 
cp.chest_pain_type,
bm.serum_cholestoral, bm.fasting_blood_sugar,bm.thal, 
pva.resting_blood_pressure,pva.resting_electrocardiographic_results,pva.maximum_heartrate_achieved,pva.exercise_induced_angina,pva.oldpeak,pva.slope_of_the_peak_exercise_st_segment,
pva.number_of_major_vessels_colored_by_flourosopy,pva.real_data,
hah.heart_attack
FROM patient p
LEFT JOIN chest_pain cp
ON p.id = cp.patient_id
LEFT JOIN blood_metrics bm
ON p.id = bm.patient_id
LEFT JOIN pressure_vessels_angina pva
ON p.id = pva.patient_id
LEFT JOIN heart_attack_history hah
ON p.id = hah.patient_id
WHERE pva.real_data <> 'Evgeny likes white wine for lunch and red wine for dinner' or pva.real_data is NULL
ORDER BY p.id ASC;
'''

# Import with pandas
df_sqlalchemy = pd.read_sql(query_string, db)
df_sqlalchemy.head()

In [ ]:
if not os.path.exists("./data"):
    os.mkdir("data")

In [ ]:
# Save dataframe as .csv file
df_sqlalchemy.to_csv("data/heart_data.csv", index=False)

In [ ]:
df = pd.read_csv('data/heart_data.csv')

In [ ]:
df.head()

In [ ]:
#Print the shape of the data
print('Heart dataset')
print('==================')
print('# observations: {}'.format(df.shape[0]))
print('# features:     {}'.format(df.shape[1]-1))

### Exploring the data

In [ ]:
df.columns

In [ ]:
# Checking for type of data present in the data frame
df.info()

In [ ]:
# Finding the descriptive statistics
df.describe().T

In [ ]:
# Checking for unique values
df.nunique()

In [ ]:
#Checking for null values in each column
df.isnull().mean()

In [ ]:
# Checking for duplicates

def check_duplicates(data):
    has_dup = data.duplicated()
    true_dup = np.where(has_dup == True)
    if len(true_dup[0]) > 0:
        print("Data has", len(true_dup[0]), "duplicates")
    else:
        print("No duplicates found !!!")

check_duplicates(df)

In [ ]:
# Checking for data imbalance
df['heart_attack'].value_counts()

In [ ]:
# Plot distribution of features 
features = df.columns.tolist()
features.remove('heart_attack')

fig,ax = plt.subplots(8,3,figsize=(34,30))
count = 0
for item in features:
    sns.histplot(df[item], kde=True, ax=ax[int(count/3)][count%3], color='#33658A').set(title=item, xlabel='')
    count += 1
ax.flat[-1].set_visible(False)
fig.tight_layout(pad=3)

### Cleaning the data

In [ ]:
df.columns

In [ ]:
# Renaming the column names

df.rename(columns = {'fasting_blood_sugar' : 'fasting_bs',
          'resting_blood_pressure' : 'resting__bp',
          'resting_electrocardiographic_results' : 'electrocardiography',
          'maximum_heartrate_achieved' : 'max_heartrate',
          'exercise_induced_angina' : 'exercise_ind_angina',
          'slope_of_the_peak_exercise_st_segment' : 'slope_of_peak_exercise',
          'number_of_major_vessels_colored_by_flourosopy' : 'major_vessels_colored'},
          inplace = True)

In [ ]:
# Checking for Outliers without heart attack column
df_outliers = df[['age', 'sex', 'chest_pain_type', 'serum_cholestoral', 'fasting_bs', 'thal', 'resting__bp', 'electrocardiography', 'max_heartrate',
       'exercise_ind_angina', 'oldpeak', 'slope_of_peak_exercise',
       'major_vessels_colored', 'real_data']].copy()

df_outliers.plot(kind='box', subplots=True, layout=(8,3), figsize=(34,30))
plt.show() #

## Analysing the data


In [ ]:
# Plotting the target variable
plt.title('heart Attack')
sns.countplot(x=df.heart_attack);

The dataset is well balanced - not deviating by much from a 50%-%50 target variable split

In [ ]:
df.real_data = np.where(df.real_data == 'real data', 'real data','null')

It is not clear whether NULL values in the column real_data is legitimate data or not. In order to check this we could 
compare the distributions of the other variables as a function of real_data.  
Check the distributions appearing on the diagonal of the below plot. Do those look similar or different?

In [ ]:
sns.pairplot(df, hue="real_data", height=2,);

The distributions look very similar - they are overlapping, peaking at the same values. Formally we could test whether those distributions are identical, but for now visual inspection is enough to accept NULL values as legitimate.

Now let's inspect how the variables differ depending on our target variable - 'heart attack'

In [ ]:
sns.pairplot(df, hue="heart_attack", height=2);

There are some significant differences now - some of the variables with big differences seem to be chest_pain_type, thal, resting_electrocardiographic_results.

In [ ]:
# Correlation heatmap

fig, ax = plt.subplots(figsize=(12,8))

# Create a new DataFrame that only includes the numerical variables
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# Compute correlations
correlations = df_numeric.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(correlations)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(140, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio

sns.heatmap(correlations, mask=mask, cmap=cmap, vmax=1, annot=True,
            linewidths=1, cbar_kws={"shrink": .7}, ax=ax);

#### Preprocessing of the Data before Modelling

In [ ]:
#we have a very large number of NaN values. 
df.dropna().info()

In [ ]:
drop_mask = df.drop(columns='real_data').isnull().any(axis=1)

# Filter out data using a negative mask and assign it a new, separated df
df = df[~drop_mask].copy()

df.info()


Columns chest_pain_type, thal, resting_electrocardiographic_results and slope_of_the_peak_exercise_st_segment are currently encoded as numeric, but looking at the plot above it is clear that they represent categorical information rather than numeric. Let's convert them to categories. In the absence of domain expertise we will continue treating number_of_major_vessels_colored_by_fluoroscopy as numeric.

In [ ]:
df.real_data = df.real_data.replace('null', 0)

In [ ]:
df

In [ ]:
df.chest_pain_type = df.chest_pain_type.astype('category')
df.thal = df.thal.astype('category')
df.electrocardiography = df.electrocardiography.astype('category')
df.slope_of_peak_exercise = df.slope_of_peak_exercise.astype('category')

In [ ]:
#Changing categorical values
df = pd.get_dummies(df)

### Train test split

In [ ]:
# Defining X and y
features = df.columns.tolist()
features.remove('patient_id')
features.remove('heart_attack')

X = df[features]
y = df.heart_attack

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

In [ ]:
# Defining X and y
features_with_id = df.columns.tolist()
features_with_id.remove('heart_attack')

X = df[features_with_id]
y = df.heart_attack

# Splitting the dataset
X_train_with_id, X_test_with_id, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Check the shape of the data sets
print("X_train_with_id:", X_train_with_id.shape)
print("y_train:", y_train.shape)
print("X_test_with_id:", X_test_with_id.shape)
print("y_test:", y_test.shape)

### Predictive Modelling

### Logistic regression using non-scaled data

In [ ]:
# Logistic Regression without the id field
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_train = log_reg.predict(X_train)
y_pred = log_reg.predict(X_test)

In [ ]:
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)

In [ ]:
# Logistic Regression with the id field
log_reg_with_id = LogisticRegression(max_iter=1000)
log_reg_with_id.fit(X_train_with_id, y_train)

y_pred_train_with_id = log_reg_with_id.predict(X_train_with_id)
y_pred_with_id = log_reg_with_id.predict(X_test_with_id)

In [ ]:
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train_with_id), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred_with_id), 2))
print("--------"*10)

Including the patient_id increases the performance of the model to 1! This makes very little sense, let's try to understand what is happening here.


In [ ]:
sns.kdeplot(data=df, x='patient_id', hue='heart_attack', fill=True)


Seems like the distributions of heart attacks are not overlapping much on the id scale. That is suspicious. Let's check it a bit closer


In [ ]:
_=sns.scatterplot(data = df,x='patient_id',y = 'heart_attack')


In [ ]:
#Mimimum id with heart_attack = 0
df.query('heart_attack == 1').patient_id.min()

In [ ]:
#Mimimum id with heart_attack = 0
df.query('heart_attack == 0').patient_id.min()

In [ ]:
#Mimimum id with heart_attack = 0
df.query('heart_attack == 1 and patient_id > 165').patient_id.min()

It looks like patients with ids from o to 165 all had heart attacks. From id 165 to 303 no one had a heart attack and for ids higher than 303 patient ids are no longer indicative of whether the patient had a heart attack or not. The person inputting the data into the database must have done this for some reason, but is there a reason to believe that the ids will follow the same pattern in the future?

No there isn't any reason to believe that - most probably after patient id 303 whether the patient had a heart attack or not will have nothing to do with the patient's id. 

Column id in our case provides an example of a phenomena called 'leakage' - "use of information in the model training process which would not be expected to be available at prediction time, causing the predictive scores (metrics) to overestimate the model's utility when run in a production environment".

Leakage can be difficult to detect as it affects both the training and the test set (as they are usually drawn from the same data set). 

https://en.wikipedia.org/wiki/Leakage_(machine_learning)

### Logistic regression using scaled data

#### Scaling with Standard scalar

In [ ]:
col_scale = ['age', 'sex', 'serum_cholestoral', 'fasting_bs',
       'resting__bp', 'max_heartrate',
       'exercise_ind_angina', 'oldpeak',
       'major_vessels_colored']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[col_scale])
X_test_scaled = scaler.transform(X_test[col_scale])

print(X_train_scaled.shape)
print(X_test_scaled.shape)

In [ ]:
# Concatenating scaled and dummy columns 
X_train_preprocessed = np.concatenate([X_train_scaled, X_train.drop(col_scale, axis=1)], axis=1)
X_test_preprocessed = np.concatenate([X_test_scaled, X_test.drop(col_scale, axis=1)], axis=1)

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_preprocessed, y_train)

y_pred_train = log_reg.predict(X_train_preprocessed)
y_pred = log_reg.predict(X_test_preprocessed)

In [ ]:
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, cmap="YlGnBu", annot=True, fmt='d');

### Scaling with MinMaxScaler (Data Normalization)

In [ ]:
# define min max scaler
minmax_scaler = MinMaxScaler()

X_train_scaled = minmax_scaler.fit_transform(X_train[col_scale])
X_test_scaled = minmax_scaler.transform(X_test[col_scale])

print(X_train_scaled.shape)
print(X_test_scaled.shape)

In [ ]:
# Concatenating scaled and dummy columns 
X_train_preprocessed2 = np.concatenate([X_train_scaled, X_train.drop(col_scale, axis=1)], axis=1)
X_test_preprocessed2 = np.concatenate([X_test_scaled, X_test.drop(col_scale, axis=1)], axis=1)

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_preprocessed2, y_train)

y_pred_train_norm = log_reg.predict(X_train_preprocessed2)
y_pred_norm = log_reg.predict(X_test_preprocessed2)

In [ ]:
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train_norm), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred_norm), 2))
print("--------"*10)

In [ ]:
# cm = confusion_matrix(y_test, y_pred_norm)
# sns.heatmap(cm, cmap="YlGnBu", annot=True, fmt='d');

### Logistic Regression with Randomsearch CV

In [ ]:
#what parameters does logistic regression has?
logistic = LogisticRegression()
logistic.get_params().keys()

 'n_jobs' is a parameter in RandomizedSearchCV and GridSearchCV which controls the number of parallel jobs to run during the search process. Here we use n_jobs=1 which means the search will be performed using a single job, running sequentially. If we use n_jobs = -1, it allows the search to utilize multiple CPU cores or processors, enabling parallel execution of the search process. This can speed up the search when working with large datasets or complex models.

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)


# define search space
param_grid = { "solver" : ['newton-cg', 'lbfgs', 'liblinear'],
               "penalty" : ['none', 'l1', 'l2', 'elasticnet'],
               "C" : loguniform(1e-5, 100)}


# define Random search
Random_search = RandomizedSearchCV(log_reg, param_grid, n_iter=500, scoring='accuracy', n_jobs=1, cv=cv, random_state=1)


# execute Random search
Random_search.fit(X_train_preprocessed, y_train)

In [ ]:
y_pred_train_RS = Random_search.predict(X_train_preprocessed)
y_pred_RS = Random_search.predict(X_test_preprocessed)

print("Tuned hpyerparameters :(best parameters) ",Random_search.best_params_)
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred_RS), 2))

In [ ]:
#cm = confusion_matrix(y_test, y_pred_RS)
#sns.heatmap(cm, cmap="YlGnBu", annot=True, fmt='d');

### Logistic regression with GridSearchCV

https://medium.com/@jackstalfort/hyperparameter-tuning-using-grid-search-and-random-search-f8750a464b35

we need to generate the different hyperparameter values. For C we can use np.logspace, which takes the endpoints of a range, generates num numbers evenly spaced in that range, and then takes another number (10 is the default) and raises it to each number it generated.
To generate the different types of penalties we just make a list with them
For the solvers, since we are using both penalties, we can only use liblinear and saga. These are also in a list

In [ ]:
C = np.logspace(0, 4, num=10)
penalty = ['l1', 'l2']
solver = ['liblinear', 'saga']

hyperparameters = dict(C=C, penalty=penalty, solver=solver)

gridsearch = GridSearchCV(log_reg, hyperparameters, scoring='accuracy',
                  cv=5, verbose=5, n_jobs=1)
gridsearch.fit(X_train_preprocessed,y_train)

In [ ]:
y_pred_train_GS = gridsearch.predict(X_train_preprocessed)
y_pred_GS = gridsearch.predict(X_test_preprocessed)

print("Tuned hpyerparameters :(best parameters) ",gridsearch.best_params_)
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred_GS), 2))

In [ ]:
# cm = confusion_matrix(y_test, y_pred_GS)
# sns.heatmap(cm, cmap="YlGnBu", annot=True, fmt='d');